## Import Libraries

In [1]:
# import system libs
import os
import time
# import data handling tools
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import regularizers

import kagglehub
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

## Class for Loading and Splitting Datasets

In [2]:
class EyeDiseaseDataset:
    def __init__(self, dataDir):
        self.data_dir = dataDir

    def dataPaths(self):
        filepaths = []
        labels = []
        folds = os.listdir(self.data_dir)
        for fold in folds:
            foldPath = os.path.join(self.data_dir, fold)
            filelist = os.listdir(foldPath)
            for file in filelist:
                fpath = os.path.join(foldPath, file)
                filepaths.append(fpath)
                labels.append(fold)
        return filepaths, labels

    def dataFrame(self, files, labels):

        Fseries = pd.Series(files, name='filepaths')
        Lseries = pd.Series(labels, name='labels')
        return pd.concat([Fseries, Lseries], axis=1)

    def split_(self):
        files, labels = self.dataPaths()
        df = self.dataFrame(files, labels)
        strat = df['labels']
        trainData, validData = train_test_split(df, train_size=0.8, shuffle=True, random_state=42, stratify=strat)

        return trainData, validData

In [3]:
# Download latest version
path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'eye-diseases-classification' dataset.
Path to dataset files: /kaggle/input/eye-diseases-classification


In [4]:
dataDir='/kaggle/input/eye-diseases-classification/dataset'

In [5]:
dataSplit = EyeDiseaseDataset(dataDir)
train_data, valid_data = dataSplit.split_()

## Function for Data Augmentation

In [6]:
def augment_data( train_df, valid_df, batch_size=16):

    img_size = (224,224)
    channels = 3
    color = 'rgb'


    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
              rotation_range= 30,
              horizontal_flip=True,
              vertical_flip=True,
              brightness_range=[0.5, 1.5])

    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

    train_generator = train_datagen.flow_from_dataframe(
              train_df,
              x_col='filepaths',
              y_col='labels',
              target_size=img_size,
              color_mode=color,
              batch_size=batch_size,
              shuffle=True,
              class_mode='categorical'
          )

    print("Shape of augmented training images:", train_generator.image_shape)

    valid_generator = valid_datagen.flow_from_dataframe(
              valid_df,
              x_col='filepaths',
              y_col='labels',
              target_size=img_size,
              color_mode=color,
              batch_size=batch_size,
              shuffle=True,
              class_mode='categorical'
          )

    print("Shape of validation images:", valid_generator.image_shape)

    return train_generator, valid_generator

In [8]:
train_augmented, valid_augmented = augment_data(train_data, valid_data)

Found 3373 validated image filenames belonging to 4 classes.
Shape of augmented training images: (224, 224, 3)
Found 844 validated image filenames belonging to 4 classes.
Shape of validation images: (224, 224, 3)


In [10]:
# Stop training if val_loss doesn't improve for 5 epochs
early_stop = EarlyStopping(
    monitor='val_loss',
    patience= 6,
    verbose= 1,
    restore_best_weights= True  # restore weights from the best epoch
)

# Save the best model automatically
checkpoint = ModelCheckpoint(
    'best_model.keras',  # file to save full model
    monitor='val_loss',
    save_best_only= True,
    verbose= 1
)


## Download and compile the model

In [11]:
classes = len(list(train_augmented.class_indices.keys()))

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
for layer in base_model.layers:
    layer.trainable = False

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu' , kernel_regularizer = regularizers.l2(0.01))(x)

predictions = Dense(classes, activation='softmax', kernel_regularizer = regularizers.l2(0.01))(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Fit the model

In [15]:
history = model.fit(
    train_augmented,
    epochs=50,
    validation_data=valid_augmented,
    callbacks=[early_stop, checkpoint]
    )

Epoch 1/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - accuracy: 0.7055 - loss: 3.9480
Epoch 1: val_loss improved from inf to 0.89152, saving model to best_model.keras
211/211 ━━━━━━━━━━━━━━━━━━━━ 151s 588ms/step - accuracy: 0.7057 - loss: 3.9398 - val_accuracy: 0.8092 - val_loss: 0.8915
Epoch 2/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.7836 - loss: 0.8331
Epoch 2: val_loss improved from 0.89152 to 0.70315, saving model to best_model.keras
211/211 ━━━━━━━━━━━━━━━━━━━━ 67s 317ms/step - accuracy: 0.7836 - loss: 0.8329 - val_accuracy: 0.8175 - val_loss: 0.7032
Epoch 3/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.8204 - loss: 0.6753
Epoch 3: val_loss did not improve from 0.70315
211/211 ━━━━━━━━━━━━━━━━━━━━ 65s 310ms/step - accuracy: 0.8204 - loss: 0.6753 - val_accuracy: 0.7713 - val_loss: 0.7782
Epoch 4/50
211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.8025 - loss: 0.6745
Epoch 4: val_loss did not improve from 0.70315
211/211 ━━━━━━━━━━━━━━━━━━━

In [ ]:
model = tf.keras.models.load_model("best_model.keras")

In [ ]:
from tensorflow.keras.preprocessing import image
img_path = "/content/Glaucoma.png"
img = image.load_img(img_path, target_size=(224, 224))  # نفس الحجم اللي اتدرب عليه
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # (1, 224, 224, 3) عشان batch

pred = model.predict(img_array)
print(pred)
class_labels = list(train_augmented.class_indices.keys())
print("Predicted:", class_labels[np.argmax(pred)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[0.08807027 0.00436081 0.89798677 0.00958208]]
Predicted: glaucoma
